In [1]:
# import libraries
import mariadb
import pandas as pd
from datetime import datetime
import os

In [101]:
# set the option to show all the columns
pd.set_option('display.max_columns', None)

## 1. Trip_info
(nTrips and NightEvent)

In [104]:
# pull trip_info data
cur.execute("SELECT * FROM trip_info WHERE tripStartTime >= '2022-07-01 00:00:00' AND tripStartTime <= '2024-07-01 00:00:00' ORDER BY tripStartTime desc")
trip_info_data = pd.DataFrame(cur.fetchall())
conn.commit()

trip_info_data.columns = ("trip_id", "vehicleName", "groupId", "groupName", "occupants", "tripStartLatitude", "tripStartLongitude", "tripStartTime",
                         "tripStartOffset", "tripStartAddress", "tripEndLatitude", "tripEndLongitude", "tripEndTime", "tripEndOffset", "tripEndAddress",
                         "stopTime", "distanceTravelled", "tripNumber", "tripTime", "idleTime", "hardBrakingCount", "hardCoreBrakingCount", "hardAccelerationCount",
                         "overspeedingCount", "overspeedingDuration", "corneringCount", "maximumSpeed", "averageSpeed", "fuelConsumed", "participantId", "beaconIssued",
                         "beaconDetected", "tripLocalStartDate", "tripLocalStartTime", "weekNumber", "tripLocalEndDate", "tripLocalEndTime", "SunRise", "SunSet")

In [105]:
trip_info = trip_info_data[[
 'trip_id',
 'vehicleName',
 'tripStartTime',
 'distanceTravelled',
 'tripNumber',
 'tripTime',
 'hardBrakingCount',
 'hardCoreBrakingCount',
 'hardAccelerationCount',
 'overspeedingCount',
 'corneringCount',
 'maximumSpeed',
 'averageSpeed',
 'participantId',
 'tripLocalStartDate',
 'tripLocalStartTime',
 'tripLocalEndTime',
 'SunRise',
 'SunSet']].copy()

print(trip_info.shape)

# Remove trips without movements (use distanceTravelled)
trip_info = trip_info.loc[trip_info['distanceTravelled'] > 0]
# Remove trips without time changed (use triptime)
trip_info = trip_info.loc[trip_info['tripTime'] > 0]

print(trip_info.shape)

(1054391, 19)
(973641, 19)


In [106]:
# define a function to extract year and month and convert 'year', 'month', 'vehicleName' to string
def preprocess_dataframe(trip_info):
    # Convert 'tripStartTime' to datetime
    trip_info['tripStartTime'] = pd.to_datetime(trip_info['tripStartTime'])
    
    # Extract year and month
    trip_info.loc[:,'year'] = trip_info['tripStartTime'].dt.year.astype(str)
    trip_info.loc[:,'month'] = trip_info['tripStartTime'].dt.month.astype(str)
    trip_info.loc[:,'day'] = trip_info['tripStartTime'].dt.day.astype(str)
    # Convert 'vehicleName' to string
    trip_info.loc[:,'vehicleName'] = trip_info['vehicleName'].astype(str)

    return trip_info

# apply the function to df
trip_info = preprocess_dataframe(trip_info)

In [107]:
# Create the new column 'vid' by concatenating 'vehicleName', 'year' and 'month'
trip_info = trip_info.assign(vid=trip_info['vehicleName'] + '_' + trip_info['year'] + '_' + trip_info['month'])

In [108]:
# nTrips
# Initialize target subgroup columns with zeros
trip_info.loc[:,'tripsmaller1mile'] = 0
trip_info.loc[:,'tripCount1to5miles'] = 0
trip_info.loc[:,'tripCount5to10miles'] = 0
trip_info.loc[:,'tripCount10to20miles'] = 0
trip_info.loc[:,'tripCount20plusmiles'] = 0

# Categorize trips into subgroups based on distance travelled
for index, row in trip_info.iterrows():
    distance = row['distanceTravelled']
    if distance < 1:
        trip_info.at[index, 'tripsmaller1mile'] = 1
    elif 1 <= distance < 5:
        trip_info.at[index, 'tripCount1to5miles'] = 1
    elif 5 <= distance < 10:
        trip_info.at[index, 'tripCount5to10miles'] = 1
    elif 10 <= distance < 20:
        trip_info.at[index, 'tripCount10to20miles'] = 1
    else:
        trip_info.at[index, 'tripCount20plusmiles'] = 1

In [109]:
# nightEvent
# Convert start time, end time, sunrise and sunset to string
trip_info['tripLocalStartTime'] = trip_info['tripLocalStartTime'].astype(str)
trip_info['tripLocalEndTime'] = trip_info['tripLocalEndTime'].astype(str)
trip_info['SunRise'] = trip_info['SunRise'].astype(str)
trip_info['SunSet'] = trip_info['SunSet'].astype(str)
# Convert start time, end time, sunrise and sunset to datetime format
trip_info.loc[:,'tripLocalStartTime'] = trip_info['tripLocalStartTime'].str.strip()
trip_info.loc[:,'tripLocalEndTime'] = trip_info['tripLocalEndTime'].str.strip()

trip_info.loc[:,'tripLocalStartTime'] = trip_info['tripLocalStartTime'].str[-8:]
trip_info.loc[:,'tripLocalEndTime'] = trip_info['tripLocalEndTime'].str[-8:]

trip_info.loc[:,'tripLocalStartTime'] = pd.to_datetime(trip_info['tripLocalStartTime'], format='%H:%M:%S').dt.time
trip_info.loc[:,'tripLocalEndTime'] = pd.to_datetime(trip_info['tripLocalEndTime'], format='%H:%M:%S').dt.time

trip_info.loc[:,'SunRise'] = trip_info['SunRise'].str.strip()
trip_info.loc[:,'SunSet'] = trip_info['SunSet'].str.strip()

trip_info.loc[:,'SunRise'] = trip_info['SunRise'].str[-8:]
trip_info.loc[:,'SunSet'] = trip_info['SunSet'].str[-8:]

trip_info.loc[:,'SunRise'] = pd.to_datetime(trip_info['SunRise'], format='%H:%M:%S').dt.time
trip_info.loc[:,'SunSet'] = pd.to_datetime(trip_info['SunSet'], format='%H:%M:%S').dt.time

# Function to determine if a trip is a night event
def is_day_event(row):
    return (row['tripLocalStartTime'] > row['SunRise']) and (row['tripLocalEndTime'] < row['SunSet'])

#create a new column 'night_event'
trip_info['night_event'] = trip_info.apply(lambda row: 0 if is_day_event(row) else 1, axis=1)

### 2. Monthly Mobility

In [111]:
# pull mobility_metrics_monthly data
cur.execute("SELECT * FROM mobility_metrics_monthly WHERE CASE WHEN RIGHT(yearMonth,2) IN (10,11,12) THEN CONCAT(yearMonth,'-01') ELSE CONCAT(CONCAT(LEFT(yearMonth,5),CONCAT('0',RIGHT(yearMonth,1))),'-01') END >= '2022-07-01' AND CASE WHEN RIGHT(yearMonth,2) IN (10,11,12) THEN CONCAT(yearMonth,'-01') ELSE CONCAT(CONCAT(LEFT(yearMonth,5),CONCAT('0',RIGHT(yearMonth,1))),'-01') END <= '2024-07-01' ORDER BY yearMonth DESC")
mobility_data = pd.DataFrame(cur.fetchall())
conn.commit()

mobility_data.columns = (
 'vehicleName',
 'yearMonth',
 'tripCount',
 'homeLatitude',
 'homeLongitude',
 'radiusOfGyration',
 'radiusOfGyration_2k',
 'radiusOfGyration_3k',
 'radiusOfGyration_4k',
 'maxDistanceFromHome',
 'maxDistance',
 'numberOfUniqueDestinations',
 'randomEntropy',
 'realEntropy',
 'uncorrelatedEntropy',
 'DRIVESentropy')

# drop test vehicle
mobility_data = mobility_data[~mobility_data['vehicleName'].isin(['Ganesh B Test for Da', 'Sarah Test for Dashc', 'Study Vehicle'])]

In [112]:
mobility = mobility_data[[
'vehicleName', 
'yearMonth', 
'tripCount', 
'radiusOfGyration', 
'numberOfUniqueDestinations', 
'realEntropy', 
'randomEntropy', 
'maxDistanceFromHome', 
#'maxDistance' # has missing data, so we won't use variable here
]].copy()

In [113]:
# convert to datetime format
mobility['yearMonth'] = pd.to_datetime(mobility['yearMonth'], format='%Y-%m')

# Create 'year' and 'month' columns
mobility['year'] = mobility['yearMonth'].dt.year
mobility['month'] = mobility['yearMonth'].dt.month

# Convert 'vehicleName' to string
mobility.loc[:,'year'] = mobility['year'].astype(str)
mobility.loc[:,'month'] = mobility['month'].astype(str)
mobility.loc[:,'vehicleName'] = mobility['vehicleName'].astype(str)

# add vid as the key
# Create the new column 'vid' by concatenating 'vehicleName' and 'month'
mobility = mobility.assign(vid=mobility['vehicleName'] + '_' + mobility['year'] + '_' + mobility['month'])

In [114]:
# There are two observations for 2024-2, keeping the first occurrence
mobility = mobility.sort_values(by= ['vehicleName', 'year', 'month']) \
               .drop_duplicates(subset=['vehicleName', 'year', 'month'], keep='first')

### 3. Calculated Driving Features and Merge All Driving Features (calculated from mobility and trip_info) into one df
('nHardcoreBrake',
'nHardcornering',
'randomEntropy',
'nDaysDriven',
'nTrips_1mi',
'nTrips_1to5mi',
'radiusOfGyration',
'maxDistanceFromHome',
'maxDistance',
'numberOfUniqueDestnt',
'gender')

In [ ]:
# Check if there is one vehicle shared by one than one px
v_p_counts = trip_info.groupby('vehicleName')['participantId'].nunique()

# Filter groups where the count of unique participant IDs is greater than 1
vehicles_with_multiple_participants = v_p_counts[v_p_counts > 1]

# Print the vehicle names with multiple participant IDs
print(vehicles_with_multiple_participants)

# To maintian our data quality, all the vehicles shared by multiple px were removed
trip_info['vehicleName'] = trip_info['vehicleName'].astype(str)
# removed both of the observations when two partcipants sharing one vehicle
trip_info = trip_info[trip_info['vehicleName'] != '']
trip_info = trip_info[trip_info['vehicleName'] != '']
trip_info = trip_info[trip_info['vehicleName'] != '']
trip_info = trip_info[trip_info['vehicleName'] != '']
trip_info = trip_info[trip_info['vehicleName'] != '']
trip_info.shape

In [132]:
# remove px is 0
trip_info = trip_info[trip_info['participantId'] != '0']

In [136]:
# nDaysDriven
# Group by 'vid' and 'year_month', and count unique days
days_driven_per_month = trip_info.groupby(['participantId','year', 'month'])['day'].nunique().reset_index()
# Rename the column to 'Days_driven_per_month'
days_driven_per_month.rename(columns={'day': 'Days_driven_per_month'}, inplace=True)
# Merge this new column back into the trip_info
trip_info = pd.merge(trip_info, days_driven_per_month, on=['vid'], how='left')

In [166]:
# Calculate featured for each vehicle and each month (vid = vehicleName + year + month)
vehicle_indct = trip_info.groupby(['vid']).agg(
    TotalDist=('distanceTravelled', 'sum'), # Total distance travelled monthly during the study period
    nTrips_1mi=('tripsmaller1mile', 'sum'), # The total number of trips with a distance smaller than 1 mi 
    nTrips_1to5mi=('tripCount1to5miles', 'sum'), 
    maxDistance=('distanceTravelled', 'max'),# The maxium distance traveled in a month, not using mobility data because there is missing in mobility
    nHardcoreBrake=('hardCoreBrakingCount', 'sum'), # The total number of hardcore braking
    nHardcornering=('corneringCount', 'sum'), # The total number of hard cornering
).reset_index()

# Calculate hardcorebraking and nHardcornering per mile
vehicle_indct['nHardcoreBrake'] = vehicle_indct['nHardcoreBrake'] / vehicle_indct['TotalDist']
vehicle_indct['nHardcornering'] = vehicle_indct['nHardcornering'] / vehicle_indct['TotalDist']

# Add vehicleId back by merging
vehicle_mapping = trip_info[['participantId', 'vehicleName']].drop_duplicates()
vehicle_indct = vehicle_indct.merge(vehicle_mapping, on='participantId', how='left')

# add the nDaysDriven to vehicle_indct
vehicle_indct = pd.merge(vehicle_indct, days_driven_per_month, on=['vid'], how='left')

In [ ]:
# merge vehicle_indct and mobility to one dataframe based on the vid. 
tmp_driving_features = pd.merge(vehicle_indct, mobility, left_on='vid', right_on='vid', how='inner').reset_index(drop = True)

In [ ]:
driving_features = tmp_driving_features[['vid',
                                         'vehicleName', 
                                             'nHardcoreBrake',
                                             'nHardcornering',
                                             'Days_driven_per_month',
                                             'nTrips_1mi',
                                             'nTrips_1to5mi',
                                             'randomEntropy',
                                             'radiusOfGyration',
                                             'maxDistanceFromHome',
                                             'maxDistance',
                                             'numberOfUniqueDestinations',
                                             'year',
                                             'month']]

In [ ]:
driving_features.describe()

### 4. Static - merged in to one df - 'meds_demo_dp_pure'
PHQ-9, dp status, demographics, medications
- PHQ9: https://redcap.wustl.edu/redcap/redcap_v14.0.34/DataExport/index.php?pid=6785 
- dp status: from G on Mar 14th
- demographics: https://redcap.wustl.edu/redcap/redcap_v14.0.34/DataExport/index.php?pid=7842
- medications: from G on July 11th

##### 4.1 PHQ-9

In [59]:
input_phq9 = pd.read_csv('.../PHQ9_20240822.csv') 

In [61]:
tmp = input_phq9[[
'map_id', 
'redcap_event_name', 
'phq_totscore', 
'otdate'
]].copy()

# remove NA data
tmp.dropna(inplace=True)
# change the dtype to datetime
tmp['otdate'] = pd.to_datetime(tmp['otdate'])  

In [63]:
# Define the target date
target_date = pd.to_datetime('2022-07-01')

# Define a function to find the row of data that closest to the target date 
def closest_date(row):
    return row.loc[row['otdate'].sub(target_date).abs().idxmin()]

# Apply the function to each px(id) 
closest_rows = tmp.groupby('map_id').apply(closest_date).reset_index(drop=True)

# Create a new df with the rows with closest date
phq9 = pd.DataFrame(closest_rows)

phq9 = phq9[['map_id', 'phq_totscore']]

##### 4.2 dp status
(merge phq9 to dp, keep pure controls-non-depression and PHQ9=0)

In [66]:
input_dp = pd.read_csv('.../id_395.csv') 

In [68]:
dp = input_dp.copy()

In [70]:
# merge to dp
tmp_dp_phq = pd.merge(dp, phq9, left_on='id', right_on='map_id', how='left').reset_index(drop = True)
tmp_dp_phq = tmp_dp_phq.drop(['map_id', 'Obs'], axis=1)

# Drop if the dp = no and Total score not equals to 0
tmp_dp_phq = tmp_dp_phq[~((tmp_dp_phq['dp'] == 'no') & (tmp_dp_phq['phq_totscore'] != 0))].copy()
dp_pure = tmp_dp_phq.drop(['phq_totscore'], axis=1)

##### 4.3 Demographics
(gender, educ, age)

In [73]:
input_demo = pd.read_csv('.../Static_20240822.csv') 

In [75]:
tmp_demo = input_demo[['id', 
                       'gender',
                       'educ',
                       'birth'
                      ]].copy()

In [77]:
# Define the target date
target_date = pd.to_datetime('2022-07-01')

In [79]:
# Function to calculate age
def calculate_age(birth):
    #today = datetime.today()
    # Set today's date to 2024-09-03
    #today = datetime(2024, 9, 3)
    age = target_date.year - birth.year - ((target_date.month, target_date.day) < (birth.month, birth.day))
    return age

# Convert the 'birth' column to datetime
tmp_demo['birth'] = pd.to_datetime(tmp_demo['birth'])

# Create 'age' column by applying the calculate_age function
tmp_demo['age'] = tmp_demo['birth'].apply(calculate_age)

In [81]:
demo = tmp_demo[['id', 'gender', 'educ', 'age']].copy()
# merge demo score to driving
demo_dp_pure = pd.merge(dp_pure, demo, left_on='id', right_on='id', how='left').reset_index(drop = True)

In [93]:
demo_dp_pure["age"].mean()

73.34705882352941

In [95]:
demo_dp_pure["age"].std()

5.689460581856214

##### 4.4 Medications

In [ ]:
input_meds = pd.read_csv('.../Meds_20240911.csv') 

In [ ]:
temp_meds = input_meds.copy()
# Convert columns to numeric 
temp_meds['MAO.inhibitors'] = pd.to_numeric(temp_meds['MAO.inhibitors'])
temp_meds['SSRI.SNRI'] = pd.to_numeric(temp_meds['SSRI.SNRI'])
temp_meds['TCAs'] = pd.to_numeric(temp_meds['TCAs'])

temp_meds['Anticonvulsants'] = pd.to_numeric(temp_meds['Anticonvulsants'])
temp_meds['Antipsychotics'] = pd.to_numeric(temp_meds['Antipsychotics'])
temp_meds['Benzodiazepine'] = pd.to_numeric(temp_meds['Benzodiazepine'])
temp_meds['Non.Benzo.Hypnotics.Sedatives'] = pd.to_numeric(temp_meds['Non.Benzo.Hypnotics.Sedatives'])
temp_meds['CNS.drugs'] = pd.to_numeric(temp_meds['CNS.drugs'])
temp_meds['Opioids'] = pd.to_numeric(temp_meds['Opioids'])
temp_meds['NSAID.APAP'] = pd.to_numeric(temp_meds['NSAID.APAP'])

# caluculate Antidepressants and Total Meds
#Antidepressant = MAO.inhibitors+SSRI.SNRI+TCAs
temp_meds['Antidepressant'] = [
    1 if (temp_meds.iloc[i]['MAO.inhibitors'] == 1 or 
              temp_meds.iloc[i]['SSRI.SNRI'] == 1 or 
              temp_meds.iloc[i]['TCAs'] == 1) else 0
    for i in range(len(temp_meds))
]

#Total = Anticonvulsants + Antipsychotics + Benzodiazepine + Non_Benzo_Hypnotics + CNS_drugs +  Opioids +  Antidepressant + NSAID_APAP
temp_meds['Total']= (temp_meds['Anticonvulsants'] + temp_meds['Antipsychotics'] + temp_meds['Benzodiazepine'] + temp_meds['Non.Benzo.Hypnotics.Sedatives']
+ temp_meds['CNS.drugs'] + temp_meds['Opioids']+ temp_meds['Antidepressant'] + temp_meds['NSAID.APAP'])

In [ ]:
# Define a function to find the row closest to the target date for each map_id
def closest_date_meds(row):
    return row.loc[row['testdate'].sub(target_date).abs().idxmin()]

temp_meds['testdate'] = pd.to_datetime(temp_meds['testdate'])

# Apply the function closest_date to each group (id) using groupby
closest_rows_meds = temp_meds.groupby('id').apply(closest_date_meds)

# Create a new DataFrame with the closest rows
temp_meds_2 = pd.DataFrame(closest_rows_meds)

meds = temp_meds_2[ ['id', 'Antidepressant', 'Total'] ].drop_duplicates().reset_index(drop = True).copy()

In [ ]:
# merge meds to demo_dp_pure
meds_demo_dp_pure = pd.merge(demo_dp_pure, meds, left_on='id', right_on='id', how='left').drop_duplicates().reset_index(drop = True)

In [ ]:
# make sure both df has the same key 'vid'
id_vid = trip_info[['vid', 'participantId']].drop_duplicates().reset_index(drop = True)
id_vid['participantId'] = pd.to_numeric(id_vid['participantId'])

In [ ]:
# merge static features together
static_features = pd.merge(meds_demo_dp_pure, id_vid, left_on='id', right_on='participantId', how='left').drop_duplicates().reset_index(drop = True)

In [ ]:
meds_demo_dp_pure

#### 5. Merge both static df and Driving features to one df

In [ ]:
# check missing
check1 = static_features.isnull().sum()
print(check1)

check2 = driving_features.isnull().sum()
print(check2)

In [ ]:
# merge
df = pd.merge(static_features, driving_features, left_on='vid', right_on='vid', how='inner').drop_duplicates().reset_index(drop = True)

In [ ]:
df.shape

In [ ]:
check_df = df.isnull().sum()
print(check_df)

In [ ]:
# fill missing meds with 0
df['Total'].fillna(0, inplace=True)
df['Antidepressant'].fillna(0, inplace=True)

#### 6. Check data

In [ ]:
yes = df['dp'] == 'yes'
print(f"Number of depressed px in df: {yes.sum()}")

no = df['dp'] =='no'
print(f"Number of non-depressed px in df: {no.sum()}")

In [ ]:
# check dp description of df groupby unique id
df.groupby('participantId')['dp'].agg(lambda x: x.value_counts().index[0])
# count how many are yes and how many are no groupby unique id
df_dp_count = df.groupby('participantId')['dp'].agg(lambda x: x.value_counts().index[0]).value_counts()
print(df_dp_count)

In [ ]:
df.describe()

In [ ]:
df_final = df[[
'participantId',
'vehicleName',
# 'id',
 'dp',
 'gender',
 'educ',
 'age',
 'Antidepressant',
 'Total',
# 'vid',
 'nHardcoreBrake',
 'nHardcornering',
 'Days_driven_per_month',
 'nTrips_1mi',
 'nTrips_1to5mi',
 'randomEntropy',
 'radiusOfGyration',
 'maxDistanceFromHome',
 'maxDistance',
 'numberOfUniqueDestinations',
 'year',
 'month'
]]